In [1]:
import pandas as pd
from pandasql import sqldf
import numpy as np

from dash import Dash, html, dcc, dash_table
import dash_bootstrap_components as dbc

import plotly.express as px

In [2]:
import SimulationEngine, PLScraper, manipulate

In [3]:
lastseason_fpath="data/2022-23 PL/2023_matchday_results.csv"
lastseason=pd.read_csv(lastseason_fpath)

oldcols=['fixture.date','teams.home.name','goals.home','teams.away.name','goals.away']
newcols=['Datetime','HomeTeam','HomeGoals','AwayTeam','AwayGoals']
lastseason=manipulate.rename_cols(lastseason,oldcols,newcols)
lastseason=lastseason[newcols]
lastseason['Status']='FT'

prevnames=['Brighton','Newcastle','Tottenham','West Ham','Wolves']
newnames=['Brighton and Hove Albion', 'Newcastle United', 'Tottenham Hotspur', 'West Ham United','Wolverhampton Wanderers']
lastseason=manipulate.df_rename_vals(lastseason,['HomeTeam','AwayTeam'],prevnames,newnames)

thisseason=PLScraper.scrape_pl_results()
thisseason=manipulate.df_strings_to_numbers(thisseason,['HomeGoals','AwayGoals'])

In [4]:
query='''
    WITH FINAL AS
    (
     SELECT Datetime, HomeTeam, HomeGoals, AwayTeam, AwayGoals, Status
    FROM lastseason
    
    UNION ALL
    
    SELECT Datetime, HomeTeam, HomeGoals, AwayTeam, AwayGoals, Status
    FROM thisseason
    WHERE
    Datetime > (SELECT MAX(Datetime) FROM lastseason)
    AND Status='FT'
    )
    SELECT *
    FROM FINAL
    ORDER BY Datetime DESC;
    '''

results_df=sqldf(query)

In [5]:
def wld_to_table(wld_df):
    table=wld_df[['Team', 'W', 'D', 'L', 'GF', 'GA']].groupby(['Team']).sum()
    table['GD']=table['GF']-table['GA']
    table['Points']=3*table['W']+table['D']
    return table

def generate_league_table(season_df):
    season_df['HomeWin']=season_df['HomeGoals']>season_df['AwayGoals']
    season_df['AwayWin']=season_df['HomeGoals']<season_df['AwayGoals']
    season_df['Draw']=season_df['HomeGoals']==season_df['AwayGoals']
    
    hometable_rename_cols={'HomeTeam':'Team','HomeWin':'W','Draw':'D','AwayWin':'L','HomeGoals':'GF','AwayGoals':'GA'}
    temp_season_df=season_df.rename(columns=hometable_rename_cols)
    hometable=wld_to_table(temp_season_df)
    
    awaytable_rename_cols={'AwayTeam':'Team','AwayWin':'W','Draw':'D','HomeWin':'L','AwayGoals':'GF','HomeGoals':'GA'}
    temp_season_df=season_df.rename(columns=awaytable_rename_cols)
    awaytable=wld_to_table(temp_season_df)

    fulltable=hometable+awaytable
    
    fulltable=fulltable.sort_values(by=['Points','GD','GF'],ascending=False).reset_index()
    fulltable['Pos']=fulltable.index+1
    orderedcols=['Pos','Team','W','D','L','GF','GA','GD','Points']
    
    return fulltable[orderedcols]

In [6]:
generate_league_table(lastseason)

,Pos,Team,W,D,L,GF,GA,GD,Points
0,1,Manchester City,28,5,5,94,33,61,89
1,2,Arsenal,26,6,6,88,43,45,84
2,3,Manchester United,23,6,9,58,43,15,75
3,4,Newcastle United,19,14,5,68,33,35,71
4,5,Liverpool,19,10,9,75,47,28,67
5,6,Brighton and Hove Albion,18,8,12,72,53,19,62
6,7,Aston Villa,18,7,13,51,46,5,61
7,8,Tottenham Hotspur,18,6,14,70,63,7,60
8,9,Brentford,15,14,9,58,46,12,59
9,10,Fulham,15,7,16,55,53,2,52


In [7]:
def get_team_results(season_df):
    season_df['HomeWin']=season_df['HomeGoals']>season_df['AwayGoals']
    season_df['AwayWin']=season_df['HomeGoals']<season_df['AwayGoals']
    season_df['Draw']=season_df['HomeGoals']==season_df['AwayGoals']
    
    home_results=season_df[['HomeTeam',b]]

In [8]:
def get_team_results(season_df,team):
    thisseason['HomeWin']=thisseason['HomeGoals']>thisseason['AwayGoals']
    thisseason['AwayWin']=thisseason['HomeGoals']<thisseason['AwayGoals']
    thisseason['Draw']=thisseason['HomeGoals']==thisseason['AwayGoals']
    
    home_rename_cols={'HomeTeam':'Team','AwayTeam':'Opponent',
                      #'HomeWin':'W','Draw':'D','AwayWin':'L',
                      'HomeGoals':'GF','AwayGoals':'GA'}
    home_results=thisseason[
        ['HomeTeam','AwayTeam','HomeGoals','AwayGoals','HomeWin','AwayWin','Draw','Datetime']
    ].query(f"HomeTeam=='%s'"%(team))

    home_results['Home/Away']='Home'
    home_results['Result']=home_results[
        ['HomeWin','AwayWin','Draw']
    ].apply(lambda row: ['W','L','D'][np.argmax(row)],axis=1)
    home_results=home_results.rename(columns=home_rename_cols
                                    ).drop(columns=['HomeWin','AwayWin','Draw'])
    
    away_rename_cols={'AwayTeam':'Team','HomeTeam':'Opponent',
                      #'AwayWin':'W','Draw':'D','HomeWin':'L',
                      'AwayGoals':'GF','HomeGoals':'GA'}
    away_results=thisseason[
        ['HomeTeam','AwayTeam','HomeGoals','AwayGoals','HomeWin','AwayWin','Draw','Datetime']
    ].query(f"AwayTeam=='%s'"%(team))
    away_results['Home/Away']='Away'
    away_results['Result']=away_results[
        ['HomeWin','AwayWin','Draw']
    ].apply(lambda row: ['L','W','D'][np.argmax(row)],axis=1)
    away_results=away_results.rename(columns=away_rename_cols
                                    ).drop(columns=['Team','HomeWin','AwayWin','Draw'])
    ordered_cols=['Datetime','Opponent','Home/Away','Result','GF','GA']
    
    return pd.concat([home_results,away_results])[
        ordered_cols].sort_values(by='Datetime',ascending=False).reset_index(drop=True)

In [9]:
get_team_results(thisseason,'Chelsea')

,Datetime,Opponent,Home/Away,Result,GF,GA
0,2023-09-17 14:00:00,Bournemouth,Away,D,0,0
1,2023-09-02 15:00:00,Nottingham Forest,Home,L,0,1
2,2023-08-25 20:00:00,Luton Town,Home,W,3,0
3,2023-08-20 16:30:00,West Ham United,Away,L,1,3
4,2023-08-13 16:30:00,Liverpool,Home,D,1,1


In [10]:
testteam='Chelsea'
thisseason_table=generate_league_table(thisseason)
league_pos=int(thisseason_table['Pos'][thisseason_table.Team==testteam])
league_pts=int(thisseason_table['Points'][thisseason_table.Team==testteam])
league_gd=int(thisseason_table['GD'][thisseason_table.Team==testteam])

In [11]:
teams=pd.unique(thisseason.HomeTeam)

In [12]:
cards = [
    dbc.Card(
        [
            html.H2(f"{league_pos}", className="card-title"),
            html.P("Current League Position", className="card-text"),
        ],
        body=True,
        color="light",
    ),
    dbc.Card(
        [
            html.H2(f"{league_pts}", className="card-title"),
            html.P("Current Points", className="card-text"),
        ],
        body=True,
        color="dark",
        inverse=True,
    ),
    dbc.Card(
        [
            html.H2(f"{league_gd}", className="card-title"),
            html.P("Current Goal Difference", className="card-text"),
        ],
        body=True,
        color="primary",
        inverse=True,
    ),
]

In [13]:
def Header(name, app):
    title = html.H2(name, style={"margin-top": 5})

    return dbc.Row([dbc.Col(title, md=9)])

In [14]:
app=Dash(__name__,  external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container(
    [
        Header("Premier League Updates: %s"%testteam, app),
        html.Hr(),
        dbc.Row([dbc.Col(card) for card in cards]),
        html.Br(),
        #dbc.Row([dbc.Col(graph) for graph in graphs]),
    ],
    fluid=True,
)

if __name__ == "__main__":
    app.run_server(debug=True)

In [33]:
team='Arsenal'
team_fixture_list=(
    PLScraper.scrape_pl_fixtures()
                   .query(f"HomeTeam=='%s' | AwayTeam=='%s'"%(team,team))
                   .sort_values(by='Datetime', ascending=True).reset_index(drop=True)
)

In [34]:
team_fixture_list

,Datetime,HomeTeam,HomeGoals,AwayTeam,AwayGoals
0,2023-09-24 14:00:00,Arsenal,0.0,Tottenham Hotspur,0.0
1,2023-09-30 15:00:00,Bournemouth,0.0,Arsenal,0.0
2,2023-10-08 16:30:00,Arsenal,0.0,Manchester City,0.0
3,2023-10-21 17:30:00,Chelsea,0.0,Arsenal,0.0
4,2023-10-28 15:00:00,Arsenal,0.0,Sheffield United,0.0
5,2023-11-04 17:30:00,Newcastle United,0.0,Arsenal,0.0
6,2023-11-11 15:00:00,Arsenal,0.0,Burnley,0.0
7,2023-11-25 15:00:00,Brentford,0.0,Arsenal,0.0
8,2023-12-02 15:00:00,Arsenal,0.0,Wolverhampton Wanderers,0.0
9,2023-12-05 19:45:00,Luton Town,0.0,Arsenal,0.0
